# Santander Bank Recommender System Demo 3/3
## Testing AWS Personalize Recommender

This notebook demonstrates how to test an AWS Personalize recommender using test files stored in S3. We'll load the data, make recommendations, and evaluate the results using precision@k metrics.

## Setup and Imports

In [1]:
import pandas as pd
import boto3
import io
from tqdm import tqdm
from IPython.display import display, Markdown, HTML


# AWS Setup
s3 = boto3.client('s3')
personalize_runtime = boto3.client('personalize-runtime')

# Configuration
bucket_name = 'souhail-work-bucket'
campaign_arn = 'arn:aws:personalize:us-east-1:279988746206:campaign/SantanderCampaign'

print("Setup complete.")

Setup complete.


## Load Data from S3

In [2]:
def load_csv_from_s3(bucket, key):
    obj = s3.get_object(Bucket=bucket, Key="personalize-test-data/"+key)
    return pd.read_csv(io.BytesIO(obj['Body'].read()))


In [3]:

# Load test data
users_df = load_csv_from_s3(bucket_name, 'Users.csv')
items_df = load_csv_from_s3(bucket_name, 'Items.csv')
interactions_df = load_csv_from_s3(bucket_name, 'Interactions.csv')

print(f"Loaded {len(users_df)} users, {len(items_df)} items, and {len(interactions_df)} interactions.")

Loaded 895857 users, 24 items, and 1320939 interactions.


## Verifying that the test dans train datasets are not overlapping

In [4]:
import boto3
import pandas as pd
from io import StringIO

def load_df_from_s3(bucket, key):
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    body = obj['Body'].read().decode('utf-8')
    return pd.read_csv(StringIO(body))


# S3 bucket and file information
bucket_name = 'souhail-work-bucket'
file1_key = 'personalize-data/Interactions.csv'
file2_key = 'personalize-test-data/Interactions.csv'

# Load DataFrames from S3
df1 = load_df_from_s3(bucket_name, file1_key)
df2 = load_df_from_s3(bucket_name, file2_key)

In [5]:
df3 = pd.concat([df1, df2], ignore_index=True).drop_duplicates()

In [6]:
len(df1) + len(df2) ==len(df3)

True

## Define Helper Functions

In [15]:
def get_recommendations(user_id, num_recommendations=10):
    try:
        response = personalize_runtime.get_recommendations(
            campaignArn=campaign_arn,
            userId=str(user_id),
            numResults=num_recommendations
        )
        return [item['itemId'] for item in response['itemList']]
    except Exception as e:
        print(f"Error getting recommendations for user {user_id}: {str(e)}")
        return []

def precision_at_k(actual, predicted, k):
    act_set = set(actual[:k])
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / float(k)
    return result

print("Helper functions defined.")

Helper functions defined.


In [16]:
# Select top 1000 users with most interactions
user_interaction_counts = interactions_df['USER_ID'].value_counts()
top_1000_users = user_interaction_counts.nlargest(1000).index.tolist()

# Filter users_df to include only these 1000 users
users_df_filtered = users_df[users_df['USER_ID'].isin(top_1000_users)]

print(f"Selected {len(users_df_filtered)} users with the most interactions.")

Selected 5261 users with the most interactions.


## Prepare Data for Testing

In [17]:
# Get actual items for each user
user_actual_items = interactions_df.groupby('USER_ID')['ITEM_ID'].agg(list).to_dict()

print(f"Prepared actual items for {len(user_actual_items)} users.")

Prepared actual items for 216100 users.


## Test the Recommender

In [6]:
activate_cell = False

if activatevate_cell:
    results = []
    for user_id in tqdm(users_df_filtered['USER_ID']):
        actual_items = user_actual_items.get(user_id, [])
        recommended_items = get_recommendations(user_id)
        
        precision_5 = precision_at_k(actual_items, recommended_items, 5)
        precision_10 = precision_at_k(actual_items, recommended_items, 10)
        
        results.append({
            'user_id': user_id,
            'actual_items': actual_items,
            'recommended_items': recommended_items,
            'precision@5': precision_5,
            'precision@10': precision_10
        })
    
    results_df = pd.DataFrame(results)
    print("Testing complete.")
else:
    print("Recommender deactivated for cost purposes, latest testing completed: 100%|██████████| 5261/5261 [02:32<00:00, 34.46it/s]
Testing complete.")

100%|██████████| 5261/5261 [02:32<00:00, 34.46it/s]

Testing complete.


## Analyze Results

In [7]:
avg_precision_5 = results_df['precision@5'].mean()
avg_precision_10 = results_df['precision@10'].mean()

print(f"Average Precision@5: {avg_precision_5:.4f}")
print(f"Average Precision@10: {avg_precision_10:.4f}")

# Save results to S3
csv_buffer = io.StringIO()
results_df.to_csv(csv_buffer, index=False)
s3.put_object(Bucket=bucket_name, Key='recommendation_results.csv', Body=csv_buffer.getvalue())
print("Results saved to 'recommendation_results.csv' in S3 bucket.")

Average Precision@5: 0.2734
Average Precision@10: 0.5064
Results saved to 'recommendation_results.csv' in S3 bucket.


### Interpretation of the results:

Precision@5: means that, on average, 27% of the top 5 recommended items for each user were actually interacted with by that user in the test set.
Precision@10: means that, on average, 50% of the top 10 recommended items for each user were actually interacted with by that user in the test set.

## Analyze Recommendations for Specific Users

Example user ids: 59401, 199855, 504959, 251965

In [7]:
results_df = load_csv_from_s3(bucket_name, 'recommendation_results.csv')

In [19]:
def analyze_user_recommendations(user_id, results_df, items_df, users_df):
    user_result = results_df[results_df['user_id'] == user_id].iloc[0]
    user_data = users_df[users_df['USER_ID'] == user_id].iloc[0]

    display(Markdown(f"# Recommendation Analysis for User ID: {user_id}"))
    
    display(Markdown("## User Profile"))
    user_profile = pd.DataFrame({
        'Attribute': ['Age', 'Customer Tenure', 'Income', 'New Customer', 'Segment', 
                      'Employee Index', 'Country of Residence', 'Sex', 
                      'Customer Activity Index', 'Province', 'Account Age (Days)'],
        'Value': [user_data['AGE'], user_data['CUSTOMER_TENURE'], 
                  f"${user_data['INCOME']:.2f}", 'Yes' if user_data['NEW_CUSTOMER'] else 'No', 
                  user_data['SEGMENT'], user_data['EMPLOYEE_INDEX'], 
                  user_data['COUNTRY_RESIDENCE'], user_data['SEX'], 
                  user_data['CUSTOMER_ACTIVITY_INDEX'], user_data['PROVINCE_NAME'], 
                  user_data['ACCOUNT_AGE_DAYS']]
    })
    display(user_profile.style.set_properties(**{'text-align': 'left'}))

    display(Markdown("## Important Note"))
    display(Markdown("This user is from the test dataset. Their interactions were not used to train the recommender system, "
                     "ensuring that these recommendations are genuinely new and not based on known preferences."))

    display(Markdown("## Precision Metrics"))
    metrics = pd.DataFrame({
        'Metric': ['Precision@5', 'Precision@10'],
        'Value': [f"{user_result['precision@5']:.4f}", f"{user_result['precision@10']:.4f}"]
    })
    display(metrics.style.set_properties(**{'text-align': 'left'}))

    display(Markdown("### Explanation of Precision Metrics"))
    display(Markdown("Precision@K measures the proportion of recommended items in the top-K that are relevant.\n"
                     "- **Precision@5**: Out of the top 5 recommended items, this fraction was actually relevant to the user.\n"
                     "- **Precision@10**: Out of the top 10 recommended items, this fraction was actually relevant to the user.\n"
                     "A higher value indicates better recommendation accuracy."))

    item_names = {
        'ind_ahor_fin_ult1': 'Savings Account', 'ind_aval_fin_ult1': 'Guarantees',
        'ind_cco_fin_ult1': 'Current Accounts', 'ind_cder_fin_ult1': 'Derivatives Account',
        'ind_cno_fin_ult1': 'Payroll Account', 'ind_ctju_fin_ult1': 'Junior Account',
        'ind_ctma_fin_ult1': 'Más Particular Account', 'ind_ctop_fin_ult1': 'Particular Plus Account',
        'ind_ctpp_fin_ult1': 'Particular Account', 'ind_deco_fin_ult1': 'Short-term Deposits',
        'ind_deme_fin_ult1': 'Medium-term Deposits', 'ind_dela_fin_ult1': 'Long-term Deposits',
        'ind_ecue_fin_ult1': 'e-Account', 'ind_fond_fin_ult1': 'Funds',
        'ind_hip_fin_ult1': 'Mortgage', 'ind_plan_fin_ult1': 'Pensions',
        'ind_pres_fin_ult1': 'Loans', 'ind_reca_fin_ult1': 'Taxes',
        'ind_tjcr_fin_ult1': 'Credit Card', 'ind_valo_fin_ult1': 'Securities',
        'ind_viv_fin_ult1': 'Home Insurance', 'ind_nomina_ult1': 'Payroll',
        'ind_nom_pens_ult1': 'Pensions', 'ind_recibo_ult1': 'Direct Debit'
    }

    recommended_items = list(dict.fromkeys(user_result['recommended_items'].strip('[]').replace("'", "").split(', ')))
    actual_items = list(dict.fromkeys(user_result['actual_items'].strip('[]').replace("'", "").split(', ')))

    display(Markdown("## Recommended Items Analysis"))

    display(Markdown("### Correctly Recommended Items (with rankings)"))
    correct_items = [f" {item_names.get(item, item)} ({rank})" for rank, item in enumerate(recommended_items, 1) if item in actual_items]
    display(Markdown("\n".join(correct_items) if correct_items else "None"))

    display(Markdown("### Incorrectly Recommended Items (with rankings)"))
    incorrect_items = [f" {item_names.get(item, item)} ({rank})" for rank, item in enumerate(recommended_items, 1) if item not in actual_items]
    display(Markdown("\n".join(incorrect_items) if incorrect_items else "None"))

    display(Markdown("### Unrecommended Items (Items in test data but not recommended)"))
    unrecommended_items = [f"- {item_names.get(item, item)}" for item in set(actual_items) - set(recommended_items)]
    display(Markdown("\n".join(unrecommended_items) if unrecommended_items else "None"))

    return user_result, user_data

In [21]:
user_result, user_data = analyze_user_recommendations(199855, results_df, items_df, users_df_filtered)

# Recommendation Analysis for User ID: 199855

## User Profile

,Attribute,Value
0,Age,43
1,Customer Tenure,182
2,Income,$85648.56
3,New Customer,No
4,Segment,02 - PARTICULARES
5,Employee Index,N
6,Country of Residence,ES
7,Sex,V
8,Customer Activity Index,1.000000
9,Province,CORDOBA


## Important Note

This user is from the test dataset. Their interactions were not used to train the recommender system, ensuring that these recommendations are genuinely new and not based on known preferences.

...

 Current Accounts (1)
 Direct Debit (2)
 Particular Plus Account (3)
 e-Account (4)
 Payroll Account (6)
 Particular Account (9)
 Credit Card (10)

### Incorrectly Recommended Items (with rankings)

 Junior Account (5)
 Long-term Deposits (7)
 Taxes (8)

### Unrecommended Items (Items in test data but not recommended)

- Mortgage
- Payroll
- Securities
- Pensions

## Conclusion

This notebook has demonstrated how to test an AWS Personalize recommender using test files stored in S3. We've loaded the data, made recommendations for each user, calculated precision metrics, and provided a way to analyze individual user recommendations.

To further improve this analysis, you might consider:
1. Implementing additional evaluation metrics (e.g., NDCG, MAP)
2. Analyzing recommendation performance across different user segments
3. Visualizing the results using matplotlib or seaborn
4. Comparing results against a baseline recommender (e.g., popularity-based)

,USER_ID,AGE,CUSTOMER_TENURE,INCOME,NEW_CUSTOMER,SEGMENT,EMPLOYEE_INDEX,COUNTRY_RESIDENCE,SEX,CUSTOMER_ACTIVITY_INDEX,PROVINCE_NAME,ACCOUNT_AGE_DAYS
347500,199855,43,182,85648.56,0.0,02 - PARTICULARES,N,ES,V,1.0,CORDOBA,5365.0
498192,199855,43,184,85648.56,0.0,02 - PARTICULARES,N,ES,V,1.0,CORDOBA,5365.0
693243,199855,43,179,85648.56,0.0,02 - PARTICULARES,N,ES,V,1.0,CORDOBA,5365.0
735693,199855,43,177,85648.56,0.0,02 - PARTICULARES,N,ES,V,1.0,CORDOBA,5365.0
